# New cross-model distance

In [3]:
from koselleck import *

In [4]:
cdist('journeys','1750-1755',neighbors=['virtue','truth'], num_proc=1)

Measuring cosine distances across runs [x1]: 100%|██████████| 10/10 [00:00<00:00, 10.99it/s]


cdist
word     neighbor period    run          
journeys virtue   1750-1755 1    1.254887
         truth    1750-1755 1    1.311167
         virtue   1750-1755 2    1.104931
         truth    1750-1755 2    1.203024
         virtue   1750-1755 3    1.222848
         truth    1750-1755 3    1.287891
                            4    1.152832
         virtue   1750-1755 4    1.203968
                            5    1.120570
         truth    1750-1755 5    1.247197
         virtue   1750-1755 6    1.059626
         truth    1750-1755 6    1.073174
         virtue   1750-1755 7    1.208020
         truth    1750-1755 7    1.360738
         virtue   1750-1755 8    1.104199
         truth    1750-1755 8    1.226013
         virtue   1750-1755 9    1.301294
         truth    1750-1755 9    1.304126
         virtue   1750-1755 10   1.172972
         truth    1750-1755 10   1.306250

In [17]:
def cdist_dfpiv(dfpiv):
    o=[]
    for i1 in dfpiv.index:
        for i2 in dfpiv.index:
            if i1<i2:
                dfcmp=dfpiv.loc[[i1,i2]]
                
                dfcmpd=dfcmp.dropna(axis=1)
                
                csim=fastdist.cosine(
                    dfcmpd.iloc[0].values.astype(float),
                    dfcmpd.iloc[1].values.astype(float)
                )
                
                odx={
                    'i1':i1,
                    'i2':i2,
                    'cdist':1-csim,
                    'size1':len(dfcmp.iloc[0].dropna()),
                    'size2':len(dfcmp.iloc[0].dropna()),
                    'size':len(dfcmpd.columns)
                }
                o.append(odx)
    return pd.DataFrame(o)
            

In [18]:
def to_lnm(word1_period1,word2_period2,k=25,num_proc=1,cache=True,progress=True):
    
    word1,period1=word1_period1.split('_') if type(word1_period1)==str else word1_period1
    word2,period2=word2_period2.split('_') if type(word2_period2)==str else word2_period2

    qstr=f'{word1}_{period1},{word2}_{period2},k={k}'
    if cache:
        with get_veclib('lnm') as vl:
            if qstr in vl:
                return vl[qstr]
    
    dfnbr1=to_nbr(word1,period1,num_proc=num_proc,progress=progress)
    dfnbr2=to_nbr(word2,period2,num_proc=num_proc,progress=progress)
    
    # filter words in both models
    dfmetanbr=dfnbr1.iloc[:k].append(dfnbr2.iloc[:k])
    metaneighb=list(set(dfmetanbr.index.get_level_values('neighbor')))
    
    # dists
    dfcdist1=cdist(word1,period1,neighbors=metaneighb,progress=progress,num_proc=num_proc)
    dfcdist2=cdist(word2,period2,neighbors=metaneighb,progress=progress,num_proc=num_proc)
    dfcdist=dfcdist1.append(dfcdist2)
    
    odf=dfcdist.reset_index()
    odf['word_period_run']=list(zip(odf.word,odf.period,odf.run))
    dfpiv=odf.pivot('word_period_run','neighbor','cdist')
    
    dfpiv_dists = cdist_dfpiv(dfpiv)
    o=[]
    for i,row in dfpiv_dists.iterrows():
        odx={
            'word1':row.i1[0],
            'word2':row.i2[0],
            'period1':row.i1[1],
            'period2':row.i2[1],
            'run1':row.i1[2],
            'run2':row.i2[2],
            'lnm':row.cdist,
            'mneighb_size':row['size'],
            'neighb1_size':row.size1,
            'neighb2_size':row.size2,
        }
        o+=[odx]
    odf=pd.DataFrame(o)
    index_cols=['word1','word2','period1','period2','run1','run2']
    odf=odf.set_index(index_cols)
    
    if cache:
        with get_veclib('lnm',autocommit=True) as vl:
            vl[qstr]=odf
    
    return odf

In [19]:
odf=to_lnm('culture_1750-1755','culture_1850-1855')
odf

lnm  mneighb_size  \
word1   word2   period1   period2   run1 run2                           
culture culture 1750-1755 1750-1755 1    2     0.015712            43   
                                         3     0.023975            44   
                                         4     0.015423            44   
                                         5     0.015523            44   
                                         6     0.014969            43   
...                                                 ...           ...   
                1850-1855 1850-1855 7    9     0.012462            48   
                                         10    0.014058            48   
                                    8    9     0.016413            48   
                                         10    0.017602            48   
                                    9    10    0.012408            48   

                                               neighb1_size  neighb2_size  
word1   word2   period1   period2   run1 run2                              
culture culture 1750-1755 1750-1755 1    2               44            44  
                                         3               44            44  
                                         4               44            44  
                                         5               44            44  
                                         6               44            44  
...                                                     ...           ...  
                1850-1855 1850-1855 7    9               48            48  
                                         10              48            48  
                                    8    9               48            48  
                                         10              48            48  
                                    9    10              48            48  

[190 rows x 4 columns]

In [20]:
# odf.groupby(['word1','word2','period1','period2']).mean()

In [21]:
# odf=to_lnm('station_1750-1755','station_1850-1855')
# odf.groupby(['word1','word2','period1','period2']).mean()

In [22]:
# odf=to_lnm('god_1750-1755','god_1850-1855')
# odf.groupby(['word1','word2','period1','period2']).mean()

### Scaling up

In [23]:
def lnm_postproc(odf):
    odf=odf.reset_index().drop('run1',1).drop('run2',1)
    odf['period_cmp']=['within' if x==y else 'across' for x,y in zip(odf.period1,odf.period2)]
    odf_wi=odf[odf.period_cmp=='within']
    odf_ac=odf[odf.period_cmp=='across']
    lnm_wi=odf_wi.lnm
    lnm_ac=odf_ac.lnm
    ttd=ttest_ind(lnm_ac,lnm_wi)
    odx={
        'word1':odf_ac.iloc[0].word1,
        'word2':odf_ac.iloc[0].word2,
        'period1':odf_ac.iloc[0].period1,
        'period2':odf_ac.iloc[0].period2,
        
        **dict(odf_ac.mean()),
        
        'ttest_t':ttd.statistic,
        'ttest_p':ttd.pvalue
    }
    
    return odx

In [24]:
# lnm_postproc(odf)

In [25]:
def lnm_(objd):
    res=to_lnm(**objd)
    if res is None: res=pd.DataFrame()
    return res

def lnm(word_or_words1,word_or_words2=None,period_or_periods=None,k=25,num_proc=1,cache=True):
    
    ## input
    # words1?
    words1=tokenize_fast(word_or_words1) if type(word_or_words1)==str else list(word_or_words1)
    # words2?
    if word_or_words2 is None:
        words2=words1
    elif type(word_or_words2)==str:
        words2=tokenize_fast(word_or_words2)
    else:
        words2=list(word_or_words2)
    # periods?
    if period_or_periods is None:
        periods=get_default_periods()
    elif type(period_or_periods)==str:
        periods=tokenize_fast(period_or_periods)
    else:
        periods=list(period_or_periods)
        
    ## preproc
    objs = [
        dict(
            word1_period1=(w1,p1),
            word2_period2=(w2,p2),
            k=k,
            num_proc=1,
            cache=cache,
            progress=False
        )
        for p1 in periods
        for p2 in periods
        for w1,w2 in zip(words1,words2)
        if p1<p2 or (p1<=p2 and w1!=w2)
    ]
#     objs=objs[:10]#random.sample(objs,10)
    
    ## proc
    iterr=pmap_iter(
        lnm_,
        objs,
        num_proc=num_proc,
        desc='Measuring local neighb. distance across word-periods'
    )
    
    ## postproc
    o=[]
    for odf in iterr:
        odx=lnm_postproc(odf)
        if odx: o.append(odx)
    return pd.DataFrame(o).set_index(['word1','word2','period1','period2'])

In [ ]:
dflnm = lnm('virtue',num_proc=4)

Measuring local neighb. distance across word-periods [x4]:  33%|███▎      | 374/1128 [01:50<06:49,  1.84it/s]